Comparison text generations from GPT and GPT-2 to illustrate the notion of a model being skewed.

In [1]:
from transformers import pipeline, set_seed

generation_gpt = pipeline("text-generation", model="openai-gpt")
generation_gpt2 = pipeline("text-generation", model="gpt2")

/home/ssanchez/env/transformers/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/ssanchez/env/transformers/lib/python3.8/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feed

config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/479M [00:00<?, ?B/s]

{"timestamp":"2025-07-01T14:46:16.894739Z","level":"WARN","fields":{"message":"Reqwest(reqwest::Error { kind: Request, url: \"https://cas-server.xethub.hf.co/reconstruction/986657f8b77a4a690d73500aa22cbda25b96a55bd9b689e8312ebd983a7108d5\", source: hyper_util::client::legacy::Error(Connect, ConnectError(\"dns error\", Custom { kind: Uncategorized, error: \"failed to lookup address information: Name or service not known\" })) }). Retrying..."},"filename":"/home/runner/work/xet-core/xet-core/cas_client/src/http_client.rs","line_number":200}
{"timestamp":"2025-07-01T14:46:16.894830Z","level":"WARN","fields":{"message":"Retry attempt #0. Sleeping 651.203428ms before the next attempt"},"filename":"/root/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/reqwest-retry-0.7.0/src/middleware.rs","line_number":171}


generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/816k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/458k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [2]:
def model_size(model):
    return sum(t.numel() for t in model.parameters())

print(f"GPT size:{model_size(generation_gpt.model)/1000**2:.1f}M parameters")
print(f"GPT2 size:{model_size(generation_gpt2.model)/1000**2:.1f}M parameters")

GPT size:116.5M parameters
GPT2 size:124.4M parameters


In [3]:
def enum_pipeline_outputs(pipe, prompt, num_return_sequences):
    out = pipe(prompt, num_return_sequences=num_return_sequences, clean_up_tokenization_spaces=True)
    return "\n".join(f"{i+1}." + s["generated_text"] for i, s in enumerate(out))

prompt = "\nWhen they came back"
print("GPT completions:\n" + enum_pipeline_outputs(generation_gpt, prompt, 3))
print("")
print("GPT2 completions:\n" + enum_pipeline_outputs(generation_gpt2, prompt, 3))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


GPT completions:
1.
When they came back from the cemetery and said their good - byes at the gravesite. 
 at first i was so happy to have them return to me, i couldn't contain myself. with the relief we could share and the sense of relief they
2.
When they came back into the room. they were smiling at me and i thought i saw a hint of relief. we were all exhausted as we headed down the hallway and into the living room. 
 " where have you been? we've been so
3.
When they came back out to the clearing she was with, but they didn't see her. when she stopped by the river to drink from the spring, she saw a strange object floating across toward her. then, as suddenly as it had appeared,

GPT2 completions:
1.
When they came back, there was a small group of us waiting for them there as our security guard stood by them, then they began to run, kicking, slashing and trying to frighten me into silence. It took about ten minutes before they
2.
When they came back, the car was already moving.

"Ther

## Building a Custom Code Dataset
### Creating a dataset with Google BigQuery

tota la perafernalia que he hagut de fer per executar una consulta, guardar els resultats en una taula d'un dataset i passar les dades a una carpeta del bucket 'npl_transformers'.
Després he instalat el [gcloud CLI](https://cloud.google.com/sdk/docs/install) al servidor.
Llavors he pogut llançar la comana: ``gsutil -m -o "GSUtil:parellel_process_count=1" cp -r gs:<nom_del_bucket>/<carpeta>/resultados-*.json.gz /datasets/codeparrot``

## Working with Large Datasets

Loading a very large dataset is often a challenging task, in particular when the data is larger than your machine's RAM.
In our example, we have 50 GB of compressed data and about 200 GB of uncompressed data.

Thankfully Datasets has two specific features that allow you to set yourself free from RAM and hard drive space limitations: **memory mapping** and **streaming**.

### Memory mapping

Overcoming RAM limitations --> uses a mechanism of zero-copy and zero-overhead memory mapping.
Basically, each dataset is cached on the drive in a file that is a direct reflection of the content in RAM memory. Instead of loading the dataset in RAM, it opens a read-only pointer to this file and uses it as a substitue for RAM (using hard drive as an extension of RAM).

Here, we will direcly load our 50GB of compressed JSON files that we have stored locally in the *codeparrot* repository.
Decompress JSON files. Be careful, cause this uses **180GB of free disk!!!**

*delete_extracted=True* in the dataset's downloading configuration, we can make sure that we delete all the files we don't need anymore as soon as possible

In [17]:
from datasets import load_dataset, DownloadConfig, Features, Value
import os

download_config = DownloadConfig(delete_extracted=True)
dataset_path = os.path.expanduser("~/datasets/codeparrot/export_results/*.json.gz")

features = Features({
    "repo_name": Value("string"),
    "path": Value("string"),
    "copies": Value("string"),
    "size": Value("string"),
    "content": Value("string"),
    "license": Value("string")
})

dataset = load_dataset("json",
                       data_files=dataset_path, split="train",
                      download_config=download_config, features=features)

Resolving data files:   0%|          | 0/500 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/206 [00:00<?, ?it/s]

In [8]:
# !gunzip -c ~/datasets/codeparrot/export_results/resultados-000000000000.json.gz > ./uno.json

In [9]:
#from datasets import load_dataset
#
#dataset = load_dataset("json", data_files="./uno.json", split="train")
#print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['repo_name', 'path', 'copies', 'size', 'content', 'license'],
    num_rows: 38184
})


In [18]:
import psutil
print(f"Number of python files code in dataset : {len(dataset)}")
ds_size = sum(os.stat(f["filename"]).st_size for f in dataset.cache_files)
# os.stat.st_size is expressed in bytes, so we convert to GB
print(f"Dataset size (cache file) : {ds_size / 2**30:.2f} GB")
# Process.memory_ingo is expressed in bytes, so we convert to MB
print(f"RAM used: {psutil.Process(os.getpid()).memory_info().rss >> 20} MB")

Number of python files code in dataset : 18923569
Dataset size (cache file) : 97.06 GB
RAM used: 1720 MB


What if you can't free enough disk space to store the full dataset locally?

## Streaming

An alternative to scaling up the server you are using is to *stream* the dataset.

In [19]:
streamed_dataset = load_dataset("json",
                       data_files=dataset_path, split="train",
                      download_config=download_config, features=features,
                               streaming=True)

Resolving data files:   0%|          | 0/500 [00:00<?, ?it/s]

In streaming mode, the compressed JSON files will be opened and read on the fly. Out dataset is now an *IterableDataset* object. This means that we cannot access random elements of it (streamed_dataset[1264]), but we need to read it in order, for instance with ``next(iter(streamed_dataset))``

In [20]:
iterator = iter(streamed_dataset)

print(dataset[0] == next(iterator))
print(dataset[1] == next(iterator))

True
True


The original raw files are extracted and read on the fly when a new batch of examples is requested, and onñy that batch is loaded in memory.

One step further-- instead of pointing to the local dataset we can reference the dataset on the Hub, and then directly download samples without downloading the raw files locally:

In [23]:
remote_dataset = load_dataset('transformersbook/codeparrot', split='train', streaming=True)

README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/184 [00:00<?, ?it/s]